In [ ]:

from clifford.sta import * 
from pylab import * 
if True:
    
    thres= 1e-10
    is_close= lambda x,y:abs(x-y)<thres
    is_close_ish= lambda x,y:abs(x-y)<thres*1e4
    split = lambda a,B: (a|B)/(a^B)
    split_apart = lambda a,B: ((a|B)/B, (a^B)/B)
    nudge = lambda a: a+1e-5 if a == 0 else a
    
    # constants not defined by model
    m = rand()                   
    q = rand()
    hbar = rand()
    
    # basis elements in standard frame
    S = d12
    i = d123 
    I = d0123
    k = i*S
    C = I*S
    pis = pi*S  # == \pi_S
    
    
    ## note: since we dont have a nice way to write the dot notation 
    # we have set the differential (rdot) as rd ,  
    #rd = d0 + .2*d1 + .84*d2 + .3*d3 # known differential
    rd = D.randomV()            # random differential.
    rd = rd(i) + abs(rd(i))*d0  # put it on the light cone
    
    
    # differential components
    rd0,rdk,rds = rd(d0),rd(k),rd(S)
    rdsnot = rd-rds
    rd0not = rd-rd0
    
    # velocity
    Vs,Vk = rds/rd0 ,rdk/rd0
    V = Vs + Vk
    T = Vk/Vs
    c= 1#abs(V)
    
    # linear momentum 
    ls,lk  = m*Vs,m*Vk  
    l = ls+lk

    # this is compton wavelength *1/gamma^2
    rs = hbar/m*abs(Vs)* rds|S/abs(rds|S)  # position vector in S 
    
    
    # frequencies, angular momenta
    theta = (rds/rs)
    W = (rds/rs)/rd0
    K = (rds/rdk)/rs
    P = W + K
    p,ps,pk = m*V,m*Vs, m*Vk
    L = m*rs^Vs
    E = m*V**2
    
    # wavelengths
    lam   = -2*pi*S*V/W
    lam_s = -2*pi*S*Vs/W
    lam_k = -2*pi*S*Vk/W
    
    # currents
    J  = q*V/lam
    Js = q*Vs/lam_s
    Jk = q*Vk/lam_k
    
    # lorentz factor and helix space pitch 
    Gamma = V*Vs.inv()         # spinor form 
    gamma = abs(Gamma)   # lorentz factor
    phi = arctan(abs(Vk)/abs(Vs))
    Le = gamma**2 *L
    
    # electrodynamics
    M = Js*pis*rs**2
    B = m/q*W
    mu_ring = 4*pis*m*rs/q**2
    Phi= B*pis*rs**2

if True:
    ################ TESTS ############################
    
    # Velocity and Differentials
    assert(rd == rd0+rdk+rds == rds+rdsnot)
    assert(-theta/(rd0*rdk)*(rd0+rdk) == P)      # P-r relation
    assert(  -rs*P == (rds*rdsnot)/(rd0*rdk))    # P-r relation
    assert(  Vs+Vk == rd0not/rd0)
    
    # Angular Momentum
    assert(L == ps^rs == -m*rs**2*W == -m*Vs**2/W)
    assert(isclose(abs(rs),abs(L/(m*Vs)))) # 
    assert(  rs**2 == - L/W/m)             # radius, as f(L,W,m)
    assert(T==Vk/Vs == rdk/rds== 1/(K*rs))
    assert(isclose(abs(T),tan(phi)))
    assert(Gamma==1+T)
     
    # Wavelengths
    assert(  lam == 2*pi*S*(rs+1/K))    # lamda = krds one  
    assert(lam_s == 2*pi*S*rs== 2*pis*L/ps) # deBroglie
    assert(lam_k == 2*pi*S*(1/K))
    assert(isclose(abs(lam)**2 ,abs(lam_s)**2 +abs(lam_k)**2))
    assert(lam==lam_s+lam_k)
    
    
    # Currents 
    assert(Js == -Jk ==-q*W/(2*pis))
    assert(isclose(abs(J),abs(Js)))
    assert(isclose(abs(J),abs(Jk)))
    assert(J==-q*W/(2*pis)*((1+T)/(1-T)))
    assert(J==-q*W/(2*pis)*V/(-k*V*k))
    assert(tan(phi)*(Vk/Vs)(2).normal() ==Vk/Vs)
    
    
    # Lorentz factor 
    assert(V**2 == Vs**2 +Vk**2)
    assert(Gamma==V*Vs.inv() == 1/(1-Vk*V.inv()))
    assert(is_close(gamma,1/sqrt(1-abs(Vk**2/V**2))))
    assert(is_close(gamma,1/cos(phi)))
       
    # Energy
    assert(E == m*V**2 == -L*W + m*Vk**2) # energy momentum
    assert(E ==-Le*W==-gamma**2*L*W)  
    assert(m*Vs**2 == -L*W )    


    # dirac equation 
    x    = D.randomV()
    psi  = e**(P*x)
    dpsi = P*psi
    A    = E/(q*Vk) 
    
    assert(Le == d0*abs(Le)*d12) # interpretation of Le as hbar 
    assert( dpsi== P*psi == (W+K)*psi == W*(1-1/Vk)*psi == -E/Le*(1-1/Vk)*psi) 
    assert(Le*dpsi -E/Vk*psi ==-E*psi ) 
    assert(1/c*Le*dpsi - q/c *E/(q*Vk)*psi == -m*c*psi)
    assert(E/(q*Vk) == m*V/q *(1+Vs/Vk))
    #assert(1/c*abs(Le)*d12*i*dpsi*i +q/c*E/(q*abs(Vk)*d3)*i*psi*i == m*c*psi*d0) #fix 
   # assert(1/c*hbar*d12*i*dpsi*i +q/c*E/(q*abs(Vk)*d3)*i*psi*i == m*c*psi*d0) # fix 
    
    #not used 
    assert(1/c*Le*dpsi == -1/c*abs(Le)*d12*i*dpsi*i*d0 == d0/c*abs(Le)*d12*dpsi)
    #assert(q/c *E/(q*Vk)*psi ==  q/c *E/(q*abs(Vk)*d3)*i*psi*i*d0== -d0*q/c *E/(q*abs(Vk)*d3)*psi )  #second part
    
    
    
    ## diracs original formula
    dpsidt = W*psi
    assert(dpsidt == -E/Le*psi )
    assert(-Le*dpsidt == (-1/gamma**2*Le*W + Vk*pk)*psi)
   # assert(Vk == -abs(V)*sqrt(1-1/gamma**2)*d3*d0) # dirac original
    #assert(-d12*hbar*dpsidt== (d0/gamma**2*m*c**2 + (c*sqrt(1-1/gamma**2))*d3*pk)*psi)
    
if True:
    # Magnetic moment 
    assert(M == -1/2*q*rs**2*W == q/(2*m)*L)
    assert(M==2*q/(2*m)*L/2)
    assert(L/M== 2*m/q)
    
    # Permeability
    assert(q*Vs*B== -rs*m*W**2)
    assert(B == mu_ring*q*W/(4*pis*rs))
    assert(mu_ring == 4*pis*m*rs/q**2 ==4/q**2*pis*L/Vs)
    
    #Flux  quantum 
    assert(L*W==2*B*M==2*Js*Phi)
    assert(Phi ==m/q*W*pis*rs**2==-pis*L/q)
    
    # Toroid
    assert(E==-L*W*(1-T**2) == -L*W*gamma**2)
    assert(-2*E/(Js**2*lam_k)==4*pis*m*rs/q**2*(T-1/T))
    assert(4*pis*m*rs/q**2*T.inv()==-2*L*W/(Js**2*lam_k))
    
    
    # Unfinished
    ## symmetries
    assert(-d0*P*d0 == -W + K) # T 
    assert(-i*P*i   == +W - K) # P?
    assert(-I*P*I   == -W - K) # C?
    assert(split(P*I,d0)== split(rdsnot,d0))

    ## null basis trivectords
    d0pkS = .5*(d0+d3)^S 
    d0mkS = .5*(d0-d3)^S
    Pp = (abs(W)+abs(K))*d0pkS  # probably a slicker way to do this
    Pm = (abs(W)-abs(K))*d0mkS
    #assert(P == Pm+Pp)
    

## curvature and torsion -like quantity  
# these are hard to translate because `rd` is null. 
rdd = 2*(S|rd)
rddd = 2*(S|rdd)
tau = rd^rdd^rddd
Ohm = rd^rdd 

    
    

In [ ]:
Ohm


In [ ]:
(Ohm|V) + (Ohm^V)

In [ ]:
#Differential properties of curve

 
rdt = lambda t: e**(t*S)*rd*~e**(t*S)

d(rdt,0)(0) ,2*(S|rdt(0))

In [ ]:
d(d(rdt,0),0)(0),rd

In [ ]:
rd = D.randomV()            # random differential.
rd = rd(i) + abs(rd(i))*d0  # put it on the light cone
rdd = 2*(S|rd)

assert(rd| rdd==0 )
rddd = S|rdd
Ohm = rd^rdd
Pi = Ohm^rddd

rd,rdd,Ohm,Pi
split(Ohm,d0), split(rd,d0)
Ohm

In [ ]:
split(Pi,d0)

In [ ]:
T

In [ ]:
from clifford.tools import log_rotor
# lorentz invariance 
x = D.randomV()
y = D.randomV()
R = e**(log_rotor(x/y)) # need to do sqrt on this . 
R*x*~R,y

In [ ]:
P_ = R.inv()*P*R
e**(P*y) , e**(R*(P_*x)/R)


In [ ]:
P/d0*P*d0

In [ ]:

E,m*V**2,-Le*W,Le,W

In [ ]:
R = e**(rand()*I)
(R*P/R)**2

In [ ]:
psi= lambda x: e**(P*x)
P|(D.randomV()(d03))
P/I
p

In [ ]:
abs(P)

In [ ]:
R*P/R

In [ ]:

R = D.randomRotor()
assert(is_close(R*psi/R , e**((R*P/R)|( R*x/R))))
psi_ = R*psi/R 
x_= R*x/R
W_ = R*W/R
K_ = R*K/R
P_= W_+K_
Vk_  = W_/K_
psi_ , e**(P_|x_)

In [ ]:
Js,Jk,J

In [ ]:
R=D.randomMV().even
#R = e**(rand()*I)
split(R*P/R,d0)*d0


In [ ]:
x     = D.randomV()

f= lambda x: e**(P^x)
abs(d(f)(x)- -P*f(x)) 

In [ ]:
f= lambda x: e**(P|x)
abs(d(f)(x)- P*f(x))

In [ ]:
f= lambda x: e**(P*x)
d(f)(x),P*f(x)

In [ ]:
Vk

In [ ]:
psi= lambda x: e**(P|x)
psi(4*(d3))



In [ ]:
x     = D.randomV()
kw    = tau=1e-6
f     = lambda x: e**(P^x)
abs(d(f)(x)- (-P*f(x)))


In [ ]:
x = D.randomV()
abs(d(lambda x:x)(x)-4)

In [ ]:
P

In [ ]:
x = v()
f=lambda x: e**(P|x) 
d(f,1e-5)(x).clean(.001), P*f(x)


In [ ]:
f=lambda x: e**(-P^x) 
d(f,1e-5)(x), P*f(x)

In [ ]:
f(x)

In [ ]:
 
v = D.randomV
kw    = tau=1e-8

psis = dict(psi_o = lambda x: e**(P^x),
            psi_i = lambda x: e**(P|x),
            psi_g = lambda x: e**(P*x))


figure(figsize=(10,4))
alphas = np.linspace(.010,20000,300)
for k in psis:
    psi = psis[k]
    dpsi = d(psi, tau=tau)
    x= v()
    plot(alphas, [abs(dpsi(alpha*x)+(P*psi(alpha*x))) for alpha in alphas],label=k+'+')
    plot(alphas, [abs(dpsi(alpha*x)-(P*psi(alpha*x))) for alpha in alphas],label=k+'-')

axvline(abs(rs),color='k')
legend()
tight_layout()
semilogy()

In [ ]:
p=I*P
I.inv()*p == P
p^x


### Stability of Dervative arou
nd small `x` 

In [ ]:
def dank(alpha):
    x = D.randomV()*alpha
    f = lambda x:e**(P*x)
    df= d(f=f,x=x,tau=1e-5).clean(1e-4)
    return abs(df -P*f(x)) 
alphas = np.linspace(0,20,200)
figure(figsize=(10,8))
[plot(alphas, list(map(dank,alphas)),color='k', alpha=.4) for k in range(10)];


In [ ]:
e**(P|x),e**(P*x)

In [ ]:
tau=1e-6
sum([ 1/a*f(da(x,x,a,tau=tau)) for a in [d0,d1,d2,d3]]) 

In [ ]:

inP(dfpsi)/P, inP(P*fpsi(x))/P

In [ ]:
inP = lambda x: x[d012]*d012 + x[d123]*d123
inP(dfpsi)/ inP(P*fpsi(x))

In [ ]:
y = P*fpsi(x)
abs(y[d012]*d012 / y[d123]*d123)

In [ ]:
Vk

In [ ]:
d(f=fpsi(x),x=x) , P*fpsi(x)


In [ ]:
P*fpsi(x)